In [1]:
# -*- coding: utf-8 -*-

from __future__ import print_function
import tensorflow as tf
import lib.dltool as dlt

tf.reset_default_graph()

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
print("Download Done!\n")

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Download Done!



In [2]:
# parameters
learning_rate  = 1e-3
training_iters = 100000
batch_size     = 50
display_step   = 100
logs_path      = 'TensorBoard/cnn'
model_path     = "Model/cnn_model.ckpt"

# network parameters
n_input   = 784  # input data (img shape: 28 * 28)
n_classes = 10   # total classes (0-9 digits)
dropout   = 0.25 # dropout, probability to keep units

# tf graph input
x = tf.placeholder(tf.float32, [None, n_input], name = 'InputData')
y = tf.placeholder(tf.float32, [None, n_classes], name = 'LabelData')
training = tf.placeholder(tf.bool)

In [3]:
# create model
def CNN(x, dropout, training):
    # reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # conv layer-1
    h_conv1 = dlt.layers.conv2d(x, 32, 5, activation = tf.nn.relu, name = 'h_conv1')
    h_pool1 = dlt.layers.max_pool2d(h_conv1, name = 'h_pool1')

    # conv layer-2
    h_conv2 = dlt.layers.conv2d(h_pool1, 64, 5, activation = tf.nn.relu, name = 'h_conv2')
    h_pool2 = dlt.layers.max_pool2d(h_conv2, name = 'h_pool2')

    # full connection
    # reshape conv2 output to fit fully connected layer input
    h_pool2_flat = tf.contrib.layers.flatten(h_pool2)
    h_fc1 = dlt.layers.dense(h_pool2_flat, 1024, activation = tf.nn.relu, name = 'h_fc1')

    # apply dropout
    h_fc1_drop = tf.layers.dropout(h_fc1, rate = dropout, training = training)

    # output layer, class prediction
    out = dlt.layers.dense(h_fc1_drop, n_classes, name = 'out')
    return out

In [4]:
with tf.name_scope('Model'):
    # build model
    pred = CNN(x, dropout, training)

# define loss and optimizer
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = pred))

with tf.name_scope('SGD'):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

# evaluate model
with tf.name_scope('Accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#kernel = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'name/kernel')[0]
#bias = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, 'name/bias')[0]

tf.summary.scalar("loss", cost)
tf.summary.scalar("accuracy", accuracy)

merged_summary_op = tf.summary.merge_all()

# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter(logs_path, graph = tf.get_default_graph())

step = 1
while step * batch_size < training_iters:
    batch_x, batch_y = mnist.train.next_batch(batch_size)

    # run optimization op (backprop)
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, training: True})

    if step % display_step == 0:
        # calculate batch loss and accuracy
        loss, acc, summary = sess.run([cost, accuracy, merged_summary_op], \
                                      feed_dict = {x: batch_x, y: batch_y, \
                                      training: False})

        summary_writer.add_summary(summary, step * batch_size)

        print("Iter              : %d  " %(step * batch_size))
        print("Minibatch Loss    : %.2f" %(loss))
        print("Training  Accuacy : %.2f" %(acc), "\n")

    step += 1

print("Optimization Finished!")

# Save model weights to disk
save_path = saver.save(sess, model_path)
print("Model saved in file: %s" % save_path)

Iter              : 5000  
Minibatch Loss    : 0.31
Training  Accuacy : 0.90 

Iter              : 10000  
Minibatch Loss    : 0.09
Training  Accuacy : 1.00 

Iter              : 15000  
Minibatch Loss    : 0.11
Training  Accuacy : 0.98 

Iter              : 20000  
Minibatch Loss    : 0.13
Training  Accuacy : 0.96 

Iter              : 25000  
Minibatch Loss    : 0.03
Training  Accuacy : 1.00 

Iter              : 30000  
Minibatch Loss    : 0.16
Training  Accuacy : 0.98 

Iter              : 35000  
Minibatch Loss    : 0.14
Training  Accuacy : 0.96 

Iter              : 40000  
Minibatch Loss    : 0.03
Training  Accuacy : 0.98 

Iter              : 45000  
Minibatch Loss    : 0.02
Training  Accuacy : 1.00 

Iter              : 50000  
Minibatch Loss    : 0.02
Training  Accuacy : 1.00 

Iter              : 55000  
Minibatch Loss    : 0.06
Training  Accuacy : 0.98 

Iter              : 60000  
Minibatch Loss    : 0.15
Training  Accuacy : 0.96 

Iter              : 65000  
Minibatch Los

In [6]:
sess.run(tf.global_variables_initializer())

# Restore model weights from previously saved model
saver.restore(sess, model_path)
print("Model restored from file: %s" % model_path)

# accuacy on test
print("Testing Accuracy : %.2f" %(sess.run(accuracy, feed_dict = { \
                                x: mnist.test.images,              \
                                y: mnist.test.labels,              \
                                training: False})), "\n")

print("Run the command line : ")
print("--> tensorboard --logdir=TensorBoard/cnn")
print("Then open localhost:6006 into your web browser")

sess.close()

INFO:tensorflow:Restoring parameters from Model/cnn_model.ckpt
Model restored from file: Model/cnn_model.ckpt
Testing Accuracy : 0.99 

Run the command line : 
--> tensorboard --logdir=TensorBoard/cnn
Then open localhost:6006 into your web browser
